In [14]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
from citipy import citipy
#import api key from file
from apikey import googlekey

# Configure gmaps
gmaps.configure(api_key=googlekey)

In [7]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps



Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [3]:
csvpath = os.path.join("..","Data","forecast_data.csv")

# read in WeatherPy CSV file
citydata = pd.read_csv(csvpath)
display(citydata)

,Unnamed: 0,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Qaanaaq,77.4840,-69.3632,32.11,77,97,11.68
1,1,Qasigiannguit,68.8193,-51.1922,34.70,81,40,5.68
2,2,Baruun-Urt,46.6806,113.2792,43.74,20,12,15.57
3,3,Bluff,-46.6000,168.3333,57.61,80,98,23.62
4,4,Ushuaia,-54.8000,-68.3000,43.23,82,96,11.65
...,...,...,...,...,...,...,...,...
554,554,Atar,20.5169,-13.0499,100.13,17,4,13.94
555,555,Ban Dung,17.6990,103.2596,81.64,90,100,13.09
556,556,Santiago de Cuba,20.0247,-75.8219,89.60,41,7,10.42
557,557,Payo,-3.7508,103.6372,92.39,50,66,3.00


In [4]:
citydata.Latitude.max()

78.2186

In [5]:
# plot heatmap of city humidity on google maps
citylocs = citydata[["Latitude","Longitude"]]
humidity = citydata["Humidity"]

fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(citylocs,weights=humidity,
                                     dissipating=False,max_intensity =100,
                                     point_radius=1)
fig.add_layer(humidity_layer)
display(fig)

Figure(layout=FigureLayout(height='420px'))

In [17]:
# filter city data to find cities with ideal weather conditions
idealvacation = citydata[(citydata["Humidity"] < 40)
                         & (citydata["Max Temperature"]< 85)
                         & (citydata["Max Temperature"]> 60)
                         & (citydata["Cloudiness"] < 50)
                         & (citydata["Wind Speed"]>10)
                         & (citydata["Wind Speed"]<20)]
idealvacation["Country"] = ""
for index, row in idealvacation.iterrows():
    idealvacationrow["Country"] = citipy.nearest_city(idealvacation["Latitude"].idealvacation["Longitude"]).country_code
display(idealvacation)

<ipython-input-17-81544e4ff736>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idealvacation["Country"] = ""


AttributeError: 'Series' object has no attribute 'idealvacation'

In [10]:
#for each city in citylist
    # Call google places API to find the first hotel result within 5000 meters of each city
    #save hotel name, city, and country data to an existing list or dictionary

# Specify search parameters
target_coordinates = "43.6187102, -116.2146068" # Note: these must be strings
target_search = "Hotel"
target_radius = 8000 # Note: this is in meters
target_type = "lodging" # for more options here, see https://developers.google.com/places/web-service/supported_types

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": googlekey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" # nearbysearch, textSearch, PlaceSearch are all optioins here; verify syntax!

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()

In [12]:
pprint(response)

{'html_attributions': [],
 'next_page_token': 'ATtYBwKKs7a9himyEfGQXUuQEEkiwVv6yaauvT0mJnVTS-0D9EFNwkDchUk9MngdXxxwzyn27-9oDbXmv0HicXBtl5S3XyBUw7JTQTA3RrBFReJ-66msMAPRl0_45ksTPIT1GAdqE-maGfHS-V-Mvj3vX9ezXFV-TqfQ3cqvpTJDodPzjEMNtutEhM0Ti_gQ99fEzfqQheogMlcr7cNOLBNeFW5ie65vR1HByTWk17n1derhEmU47iXem8nTxkg1gZqwYvju_fBkgZrojJcbBWiVciaZSNH4w1mLeFrHXmQkjmsqX_FYqm2UuTuB0zDYd3wt67Xf_5sMqt27lwjvo2UbGpwf2sol108rKyUgiNMBesKjS-XGnGbSbM2_wl5JElBFke8XEBddh-BUE7CLh_VciXhzo0Jn1j9X7uSlKCi9p2iGFpIfV3lkSmrr8h7YnbHv',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 43.6139805, 'lng': -116.2037792},
                           'viewport': {'northeast': {'lat': 43.61524747989272,
                                                      'lng': -116.2022788701073},
                                        'southwest': {'lat': 43.61254782010728,
                                                      'lng': -116.2049785298927}}},
              'icon': 'https://maps.gstatic

In [ ]:
#plot these hotels on the humidity heatmap with pins containing hotel name, city, and country